<img align="right" style="max-width: 200px; height: auto" src="./sfao_logo.jpg">

##  Lab 02 - Foundation Model Audit Models

Januar Schulung, Eidgenössische Finanzkontrolle (EFK), 2025

Die Labs zur **Künstlichen Intelligenz** der Januar Schulung 2025 basieren auf Jupyter Notebook. Anhand solcher Notebooks ist es möglich eine Vielzahl von Datenanalysen und statistischen Validierungen durchzuführen.

<img align="center" style="max-width: 700px" src="./lab_02_banner.png">

In diesem Lab werden wir untersuchen, wie Foundation Modelle in Prüfungsaufgaben integriert werden können, um komplexe Prozesse effizient zu automatisieren und zu analysieren. Ziel ist es, ein vortrainiertes Modell an spezifische Aufgaben anzupassen, wie z. B. die Analyse von Buchhaltungsdaten, die Anomalieerkennung und die Textauswertung von auditrelevanten Dokumenten. Dieser Ansatz baut auf dem Konzept des [**AI Co-Piloted Auditing**](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4444763), auf, bei dem Foundation Modelle vielseitige Aufgaben übernehmen und durch ihre Fähigkeit zur Sprachverarbeitung und Mustererkennung einen Mehrwert schaffen. Die Modelle agieren nicht isoliert, sondern als Werkzeuge, die den Prüfungsprozess unterstützen und die Arbeit menschlicher Prüfer ergänzen. Die folgende Abbildung veranschaulicht den Prüfungsprozess, den wir in diesem Lab mithilfe eines Foundation Modells umsetzen möchten.

<img align="center" style="max-width: 1000px" src="./audit_process.png">

Mit [**LangChain**](https://www.langchain.com/) können wir komplexe **Chain-of-Thought-Prozesse** modellieren, die es ermöglichen, Foundation Modelle gezielt für die Analyse von Buchhaltungsbuchungen, die Anomalieerkennung und die Dokumentenauswertung einzusetzen. Im Gegensatz zu CrewAI, das stärker auf agentenbasierte Architekturen ausgerichtet ist, bietet LangChain ein flexibles Werkzeugset, um Foundation Modelle auf spezifische Anforderungen im Auditbereich zu adaptieren.

Mit dem `LangChain` Framework werden wir einen **Chain-of-Thought Prozess** (wie oben dargestellt) implementieren und bereitstellen, um Buchhaltungsbuchungen gemäss dem [International Standard on Auditing (ISA) 240](https://www.ifac.org/_flysystem/azure-private/publications/files/A012%202013%20IAASB%20Handbook%20ISA%20240.pdf) zu prüfen. Durch den Chain-of-Thought Prozess wird sichergestellt, dass die einzelnen Schritte des Prüfungsprozesses logisch aufeinander aufbauen, wodurch eine klare und nachvollziehbare Struktur für die Entscheidungsfindung entsteht.

Bei etwaigen Fragen wenden Sie sich, wie immer gerne an uns via **marco (dot) schreyer (at) efk (dot) admin (dot) ch**. Wir wünschen Ihnen Viel Freude mit unseren Notebooks und Ihren revisorischen Analysen!

## Lernziele des Labs:

Nach dem heutigen Lab sollten Sie in der Lage sein:

> 1. **Erstellung von Prompts:** Definition und Organisation von Task-, Action-, Input- und Output-Prompts. 
> 2. **Chain-of-Thought Prompting:** Nutzung von Chain-of-Thought zur Strukturierung von Prüfungsaufgaben.  
> 3. **Einsatz von LangChain:** Anwendung des LangChain-Frameworks zur Verarbeitung von Prompts.  
> 4. **Prüfungsworkflow:** Umsetzung eines Chain-of-Thought Prozesses für das Journal Entry Testing. 

Am Ende des Labs werden Sie verstehen, wie Sie mit Foundation Modellen und der `LangChain` Bibliothek Chain-of-Thought Prozesse einrichtet um Prüfprozesse zunehmend zu automatisieren. 

## 1. Einrichten der Analyseumgebung

Ähnlich wie in den vorangegangenen Übungen werden wir zunächst eine Reihe von Python-Bibliotheken importieren, welche die Datenanalyse und -visualisierung ermöglichen. In dieser Übung werden wir die Bibliotheken `Langchain`, `Pandas`, `Numpy`, `Matplotlib` und `Seaborn` verwenden. Nachfolgend importieren wir die benötigten Bibliotheken durch die Ausführung der folgenden Anweisungen:

In [ ]:
# import python data science and utility libraries
import os, io, requests, warnings
from datetime import datetime

Ausschalten möglicher Warnmeldungen z.B. aufgrund von zukünftigen Änderungen der Bibliotheken:

In [ ]:
# set the warning filter flag to ignore warnings
warnings.filterwarnings('ignore')

Installation der `PyPDF` und `LangChain` Bibliotheken:

In [ ]:
! pip -qqq install PyPDF2
! pip -qqq install langchain
! pip -qqq install langchain-openai
! pip -qqq install langchain-ollama
! pip -qqq install langchain-huggingface
! pip -qqq install langchain-community
! pip -qqq install faiss-gpu

Import der `PyPDF` PDF Analyse Bibliotheken:

In [ ]:
# import pypdf document reader libraries
from PyPDF2 import PdfReader

Import der `Langchain` LLM Building Bibliotheken:

In [ ]:
# import langchain llm building libraries
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain import LLMChain

Import der `Langchain`LLM Callback Bibliotheken:

In [ ]:
# import langchain llm callback libraries
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

Import der `Langchain`LLM Chain Bibliotheken:

In [ ]:
# import langchain llm chain libraries
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

Import der `Langchain`LLM Vectorstore Bibliotheken:

In [ ]:
# import langchain llm vectorstore libraries
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

## 2. Einrichtung des Large Language Models (LLM)

In diesem Abschnitt richten wir lokal gehostete Large Language Models (LLMs) lokal ein, welche über die [**Ollama**](https://ollama.com/) Anwendung bereitgestellt werden. Hierduch ist es uns möglich verschiedene LLMs für Prüfungsaufgaben zu evaluieren. Darüber hinaus wir verwenden wir die **Langchain**-Schnittstelle `Ollama`, um auf unterschiedliche Foundation Modelle bzw. LLMs zuzugreifen. Die **Ollama**-Anwendung bietet die Möglichkeit, LLMs lokal auszuführen. Wir werden das **LLaMA3**-Modell von Meta.ai über **Ollama** einrichten und verwenden. Dies ermöglicht eine lokale Ausführung des Modells, reduziert die Latenz und verbessert den Datenschutz.

Dazu installieren wir zunächst **Ollama** in der Google Colab-Umgebung. Zuerst müssen wir unser Colab-Notebook so einrichten, dass es Befehle über die Kommandozeile unterstützt. Der nachfolgende Code installiert entsprechend die `colab-xterm`-Bibliothek:

In [ ]:
! pip -qqq install colab-xterm

Anschliessend aktivieren wir die `Colab XTerm`-Erweiterung, wodurch wir Shell-Befehle direkt im Notebook ausführen können:

In [ ]:
%load_ext colabxterm

Nun starten wir das xterm-Terminal innerhalb unserer Colab-Zelle:

In [ ]:
%xterm

Nachdem das xterm geöffnet ist, können wir mit der Installation von Ollama fortfahren. Führen Sie die folgenden Befehle im Terminal aus, um **Ollama** zu installieren:

> curl https://ollama.ai/install.sh | sh

Dieser Befehl lädt das Installationsskript von der Ollama-Website herunter und führt es aus. Das Skript übernimmt den Installationsprozess automatisch, einschließlich des Herunterladens und Installierens notwendiger Abhängigkeiten. Nach der Installation können wir den Server mit folgendem Befehl starten:

> ollama serve &

Das `&` am Ende lässt den Befehl im Hintergrund laufen, sodass das Terminal weiter genutzt werden kann.

Nach erfolgreicher Installation führen wir den Befehl `ollama list` aus, um die verfügbaren Modelle anzuzeigen:

In [ ]:
! ollama list

Unser Ziel ist es, das **LLaMA 3.2**-Modell zu verwenden, das instruktionstauglich ist und über **3 Milliarden Parameter** verfügt (Text-in/Text-out). Dieses Modell wurde für mehrsprachige Dialogfälle optimiert, einschließlich agentenbasierter Such- und Zusammenfassungsaufgaben. Wir laden das **LLaMA 3.2**-Modell in der `Google Colab`-Anwendung mit dem `ollama pull`-Befehl herunter:

In [ ]:
! ollama pull llama3.2:3b

Anschließend überprüfen wir kurz, ob das Modell erfolgreich heruntergeladen wurde:

In [ ]:
! ollama list

Perfekt! Jetzt führen wir das Modell lokal auf Ihrem Gerät mit dem Befehl `ollama run` und dem entsprechenden Modellnamen aus:

In [ ]:
# ! ollama run llama3.2:3b

Da das Modell jetzt läuft, verwenden wir die **Langchain**-Schnittstelle `ChatOpenAI`, um eine Verbindung zu dem lokal gehosteten **LLaMA 3.2**-Modell herzustellen:

In [ ]:
# initialize the local LLaMA 3.2 by meta.ai
llm = OllamaLLM(model='llama3.2:3b', temperature=0.7, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

Als Nächstes testen wir das lokale **LLaMA3**-Modell. Wir simulieren ein Szenario, in dem die KI aufgefordert wird, einen Spesenbericht eines etwas eigenwilligen Mitarbeiters zu prüfen, der möglicherweise private Ausgaben einreichen möchte. Dieser Test stellt sicher, dass das Modell korrekt reagiert:

In [ ]:
# define a fun audit prompt 
prompt = """
Imagine you're an auditor reviewing an expense report from an employee who claims all expenses are legitimate business costs. 
Here's a list of items they've submitted:

1. Office supplies - $100
2. Coffee machine - $250
3. Unicorn onesie - $45
4. 10-pound gummy bear - $30
5. "Business trip" to Disneyland - $2000
6. Sushi for team meeting - $150

Please identify which items are legitimate business expenses and which might require further investigation, with a bit of humor in your response. 
"""

Das lokale **LLaMA3**-Modell wird beauftragt, zu bestimmen, welche Posten im Spesenbericht legitim sind und welche einer genaueren Prüfung bedürfen. Das Modell sollte eine humorvolle, aber praktische Analyse der Ausgaben liefern:

In [ ]:
# test the model with this fun audit prompt
response = llm(prompt)

In diesem Abschnitt haben wir eine Möglichkeiten erkundet, Sprachmodelle in unser Prüfungstoolkit zu integrieren: die Einrichtung eines lokalen Modells mit **Ollama** und das absetzen von einfachen Prompts. Diese Tools bieten Flexibilität bei der Durchführung Foundation Model basierter Prüfungsaufgaben. Mit dieser eingerichteten Umgebung sind wir bereit, uns praktischen Anwendungen zuzuwenden, bei denen diese Modelle als Co-Piloten in einem definierten Prüfungsprozess eingesetzt werden.

## 3. Foundation Modell basierte Prüfung von Journal Entries

In diesem Abschnitt wird die praktische Anwendung des **Chain-of-Thought Promptings** im Prüfungsprozess am Beispiel des Journal Entry Testings demonstriert. Ziel ist es, die definierten Prompts zu nutzen, um strukturierte und nachvollziehbare Ergebnisse zu generieren. Der Fokus liegt auf der schrittweisen Verarbeitung der Prompts, von der Aufgabenbeschreibung bis zur Ergebnisgenerierung.

### 3.1 Definition der Chain-of-Thoughts Prompts

Das **Chain-of-Thought Prompting** beschreibt eine Technik, die es ermöglicht, komplexe Aufgaben in logisch aufeinanderfolgende Schritte zu unterteilen. Durch diese Methode kann ein Modell nicht nur Ergebnisse generieren, sondern auch seinen Denkprozess und die angewendeten Schritte erklären. Dies ist besonders wertvoll im Audit-Kontext, wo Transparenz und Nachvollziehbarkeit essenziell sind.

Im Folgenden verwenden wir das **Chain-of-Thought Prompting**, um die Fähigkeiten des Foundation Modells systematisch zu strukturieren. Hierbei werden verschiedene Arten von Prompts definiert, die jeweils unterschiedliche Aspekte der Prüfungsaufgabe ansprechen. Die folgende Untergliederung beschreibt, wie diese Prompts gestaltet werden können, um das Modell durch den **Journal Entry Testing Prüfprozess** zu führen – von der Kontextsetzung über die Datenverarbeitung bis hin zur Identifikation von Buchungsanomalien. Jedes Prompt hat eine spezifische Funktion und trägt dazu bei, den Prüfungsprozess transparent und effizient zu gestalten.

#### 3.1.1 Definition des Task-Explanation Prompts

Das nachfolgende Prompt legt den Kontext für die durchzuführende Prüfungsaufgabe fest, z. B. die Prüfung von Buchhaltungsbuchungen, Audit-Sampling oder die Analyse von Anmerkungen. Darüber hinaus wird die Rolle erläutert, die das Foundation Modell übernehmen soll. Um mehrere Prompts zu ermöglichen, enthält es einen Satz, der das Modell anweist, auf das nächste Prompt zu warten und keine Aktionen eigenständig zu initiieren.

In [ ]:
def get_task_explanation_prompt():
    return """
    You are an excellent auditor of tabular financial transactions
    als referred to as "journal entries" . Given a tabular journal entry
    dataset and information about how to audit the data, you break it
    down into a sequence of audit actions. 
    
    Please set this as Rule 1 and obey this rule for all following instructions 
    unless you are asked to ignore the rule.

    Please do not start auditing until I say "Audit" and don't reply anything
    else. Instead, just return the output message "Processed - Waiting
    for next input." and nothing else.
    """

#### 3.1.2 Definition des Action-Explanation Prompts

Das nachfolgende Prompt definiert und beschreibt die erforderlichen Co-Pilot-Aktionen, die zur Lösung einer bestimmten Aufgabe notwendig sind, z. B. Attribut-Korrelationsanalyse, Textsentiment-Analyse oder Bildsegmentierung. Die Auswahl der geeigneten Co-Pilot-Aktionen hängt von der Prüfungsaufgabe ab und ermöglicht die Vorbereitung einer notwendigen und hinreichenden Menge von Prüferaktionen zur Manipulation von Auditdaten.

In [ ]:
def get_action_explanation_prompt():
    return """
    As an auditor, your task is to examine journal entry data
    recorded in an audit client's Enterprise Resource Planning (ERP)
    system. Specifically, you are responsible for detecting two types of
    anomalous journal entries within large datasets:

    Anomalies are classified into two categories:

    1. Global Anomalies: These anomalies involve an unusual combination of
    values for any single attribute when compared to the entire dataset.
    For example, a significantly higher or lower transaction amount for a
    specific department title or a rare department title in the dataset
    would be considered a global anomaly. Global anomalies are often
    related to an attribute that stands out across the entire dataset,
    regardless of other attributes' values.

    2. Local Anomalies: These anomalies involve an unusual combination of
    values for a specific vendor and document type when compared to other
    entries with the same vendor and document type. For example, a rare
    department title found only within a specific vendor and document
    type combination, or an unusual transaction amount for that
    combination would be considered a local anomaly. Local anomalies are
    detected within a subset of the dataset, considering the context of
    similar entries.

    Keep these definitions in mind when analyzing datasets for anomalies. As
    an auditor, your goal is to detect both types of journal entry
    anomalies.

    Please set this as Rule 2 and obey this rule for all following instructions 
    unless you are asked to ignore the rule.

    Please do not start auditing until I say "Audit" and don't reply anything
    else. Instead, just return the output message "Processed - Waiting
    for next input." and nothing else.
    """

#### 3.1.3 Definition des Input-Data-Explanation Prompts

Das nachfolgende Prompt definiert die Eingabedaten, die für die Prüfungsaufgabe verarbeitet werden sollen. Es enthält eine allgemeine Beschreibung der zu verarbeitenden Datenmodalität, z. B. tabellarische, textuelle oder Bilddaten. Darüber hinaus umfasst es eine detaillierte Beschreibung der Eingabedaten, z. B. Attributnamen, Attributsemantik, Inhaltsverzeichnis und lexikalische Informationen.

In [ ]:
def get_input_data_explanation_prompt():
    return """
    Enterprise Resource Planning (ERP) systems generally store
    journal entries in a tabular database table format. This table will
    serve as the input data for analyzing global and local anomalies.
    Each row in the table represents a single journal entry, while each
    of the six columns represents a distinct journal entry attribute. The
    six columns are defined as follows:
    
    1. Fiscal_Year: The fiscal year during which the journal entry was
    posted, running from July 1st to June 30th.
    2. Document_Number: A unique identifier for each journal entry, used for
    tracking individual entries within the organization's ERP system.
    3. Department_Title: The department title used in the organization's
    finance and accounting system that denotes the department that posted
    the journal entry.
    4. Vendor_Name: The name of the individual, vendor, or entity associated
    with the journal entry.
    5. Document_Type: The classification of the journal entry within the
    organization's ERP system.
    6. Transaction_Amount: The transaction or payment amount in USD.

    The table may contain a large, yet arbitrary, number of journal entries,
    depending on the audit client's business activity. The table data is
    formatted as a comma-separated values (CSV) file. Below is a sample
    of a database table in CSV format, with the first row representing
    attribute names and the following rows representing individual
    journal entries.
    
    Input Data:
    
    Fiscal_Year, Document_Number, Department_Title, Vendor_Name, Document_Type, Transaction_Amount
    2017, CHEK17119771, 42 COMMERCE, EAT AT JOE'S, petty cash, 50.00
    2017, CHEK17119723, 26 LICENSES & INSPECTIONS, MARLENE BELL REPORTING INC., procurement, 454.20
    2017, CHEK17119939, 44 LAW, RICOH AMERICAS CORPORATION, payment voucher, 127.33
    ... 

    Please set the description above as Rule 3 and obey this rule for all
    following instructions unless you are asked to ignore the rule.
    
    Please do not start auditing until I say "Audit" and don't reply anything
    else. Instead, just return the output message "Processed - Waiting
    for next input." and nothing else.
    """

#### 3.1.4 Definition des Output-Data-Explanation Prompts

Das nachfolgende Prompt definiert die Ausgabedaten, die zur Lösung der Prüfungsaufgabe generiert werden. Es enthält eine allgemeine Beschreibung der zu generierenden Datenmodalität, z. B. tabellarische, textuelle oder Bilddaten. Darüber hinaus umfasst es eine detaillierte Beschreibung der Ausgabedaten, z. B. Warnmeldungen, Dateiformate der Ausgabe und Visualisierungen der Ergebnisse.

In [ ]:
def get_output_data_explanation_prompt():
    return """
    In general, anomaly detection systems provide two types of
    information after successfully analyzing journal entries within a
    tabular database table. The first type of information is the detected
    anomalous journal entries themselves, and the second type is an
    explanation of why each entry was identified as an anomaly. Below is
    an example of an alert message format that should be generated when a
    local or global anomaly is detected:

    Alert Message!
    Detected Global Anomaly:
    [Placeholder for the detected anomalous journal entries.]

    Dectection Rationale:
    [Placeholder for the rationale of why the journal entry was detected as an anomaly.]

    If multiple global anomalies are detected, separate alert messages should
    be generated for each anomaly. If you find you cannot find any
    anomalies, please explain the reason. 
    
    Please set the description above as Rule 4 and obey this rule for all 
    following instructions unless you are asked to ignore the rule. 
    
    Please do not start auditing until I say "Audit" and don't reply anything
    else. Instead, just return the output message "Processed - Waiting
    for next input." and nothing else.
    """

#### 3.1.5 Definition des Global-Anomaly-Example Prompts

Das nachfolgende Prompt dient der Definition von globalen Anomaliebeispielen, die für die Lösung der Prüfungsaufgabe generiert werden. Es enthält eine allgemeine Beschreibung der Datenmodalität, z. B. tabellarische, textuelle oder Bilddaten, sowie eine detaillierte Beschreibung der Beispiele, z. B. Muster, die globale Auffälligkeiten in den Daten widerspiegeln, und entsprechende Darstellungen der Ergebnisse.

In [ ]:
def get_global_anomaly_example_prompt():
    return """
    In the following I'm providing you a global anomaly detection example.
    
    Input data:
    
    Fiscal_Year, Document_Number, Department_Title, Vendor_Name, Document_Type, Transaction_Amount
    2017, CHEK17119393, 42 COMMERCE, EAT AT JOE'S, petty cash, 68.17
    2017, CHEK17119771, 42 COMMERCE, EAT AT JOE'S, petty cash, 23.85
    2017, CHEK17118549, 42 COMMERCE, EAT AT JOE'S, petty cash, 50.00
    2017, CHEK17113292, 42 COMMERCE, EAT AT JOE'S, petty cash, 84.23
    2017, CHEK17119504, 26 LICENSES & INSPECTIONS, EAT AT JOE'S, petty cash, 72.23
    2017, CHEK17137637, 42 COMMERCE, EAT AT JOE'S, petty cash, 23.03
    2017, CHEK17119893, 42 COMMERCE, EAT AT JOE'S, petty cash, 58.17
    2017, CHEK13829239, 42 COMMERCE, EAT AT JOE'S, petty cash, 67.85
    2017, CHEK17183932, 42 COMMERCE, EAT AT JOE'S, petty cash, 72.00
    2017, CHEK17323423, 42 COMMERCE, EAT AT JOE'S, petty cash, 39.28
    2017, CHEK17139393, 42 COMMERCE, EAT AT JOE'S, petty cash, 939.22
    2017, CHEK17383833, 42 COMMERCE, EAT AT JOE'S, petty cash, 94.22
    2017, CHEK17149494, 42 COMMERCE, EAT AT JOE'S, petty cash, 34.93
    2017, CHEK17393939, 42 COMMERCE, EAT AT JOE'S, petty cash, 32.82
    2017, CHEK17339594, 42 COMMERCE, EAT AT JOE'S, petty cash, 39.28
    2017, CHEK17169595, 42 COMMERCE, EAT AT JOE'S, petty cash, 939.22
    2017, CHEK17339203, 42 COMMERCE, EAT AT JOE'S, petty cash, 94.22
    2017, CHEK17139203, 42 COMMERCE, EAT AT JOE'S, petty cash, 293.93
    2017, CHEK17393829, 42 COMMERCE, EAT AT JOE'S, petty cash, 493.22
    2017, CHEK17340309, 42 COMMERCE, EAT AT JOE'S, petty cash, 64.83
    
    And here is the expected output:
    
    Alert Message!
    
    Detected Global Anomaly:
    
    2017, CHEK17119504, 26 LICENSES & INSPECTIONS, EAT AT JOE'S, petty cash, 72.23
    
    Dectection Rationale:
    The journal entry exhibting document number CHEK17119504 corresponds to a
    global anomaly. The journal entry contains the department title "26
    LICENSES & INSPECTIONS" which in unusual when compared to all other
    journal entries in the dataset.

    Please set the description above as Rule 5 and obey this rule for all 
    following instructions unless you are asked to ignore the rule. 
    
    Please do not start auditing until I say "Audit!". Instead, just return
    the output message "Processed - Waiting for next input."
    """

#### 3.1.6 Definition des Local-Anomaly-Example Prompts

Das nachfolgende Prompt definiert lokale Anomaliebeispiele, die spezifische Auffälligkeiten in den Daten identifizieren und darstellen. Es enthält eine allgemeine Beschreibung der Datenmodalität, z. B. tabellarische, textuelle oder Bilddaten, und liefert detaillierte Informationen zu den Beispielen, z. B. Auffälligkeiten auf Attributebene, Ausgabeformate und visuelle Darstellungen der identifizierten lokalen Anomalien.

In [ ]:
def get_local_anomaly_example_prompt():
    return """
    In the following I'm providing you a local anomaly detection example.
    
    Input data:
    
    Fiscal_Year, Document_Number, Department_Title, Vendor_Name, Document_Type, Transaction_Amount
    2017, CHEK17119393, 42 COMMERCE, EAT AT JOE'S, petty cash, 68.17
    2017, ACHD17994939, 28 WATER, XEROX CORPORATION, procurement, 932.32
    2017, CHEK17119771, 42 COMMERCE, EAT AT JOE'S, petty cash, 23.85
    2017, CHEK17118549, 42 COMMERCE, EAT AT JOE'S, petty cash, 50.00
    2017, CHEK17113292, 42 COMMERCE, EAT AT JOE'S, petty cash, 84.23
    2017, ACHD17939323, 28 WATER, XEROX CORPORATION, procurement, 352.29
    2017, CHEK17119504, 42 COMMERCE, EAT AT JOE'S, petty cash, 72.23
    2017, CHEK17137637, 42 COMMERCE, EAT AT JOE'S, petty cash, 23.03
    2017, ACHD17950344, 28 WATER, EAT AT JOE'S, petty cash, 292.22
    2017, CHEK17119893, 42 COMMERCE, EAT AT JOE'S, petty cash, 58.17
    2017, ACHD17830920, 28 WATER, XEROX CORPORATION, procurement, 93.92
    2017, ACHD17994949, 28 WATER, XEROX CORPORATION, procurement, 833.59
    2017, ACHD17390203, 28 WATER, XEROX CORPORATION, procurement, 26.59
    2017, CHEK13829239, 42 COMMERCE, EAT AT JOE'S, petty cash, 67.85
    2017, CHEK17183932, 42 COMMERCE, EAT AT JOE'S, petty cash, 72.00
    2017, ACHD17300393, 28 WATER, XEROX CORPORATION, procurement, 252.19
    2017, ACHD17193303, 28 WATER, XEROX CORPORATION, procurement, 68.38
    2017, CHEK17323423, 42 COMMERCE, EAT AT JOE'S, petty cash, 39.28
    2017, ACHD17930233, 28 WATER, XEROX CORPORATION, procurement, 103.39
    2017, ACHD17392022, 28 WATER, XEROX CORPORATION, procurement, 632.84
    
    And here is the expected output:

    Alert Message!
    
    Detected Local Anomaly:
    
    2017, ACHD17950344, 28 WATER, EAT AT JOE'S, petty cash, 292.22
    
    Dectection Rationale:
    The journal entry exhibting document number ACHD17950344 corresponds to a
    local anomaly. The journal entry contains the department title "28
    Water" which is usually not observed with the vendor name "EAT AT
    JOE'S" and document type "petty cash".

    Please set the description above as Rule 6 and obey this rule for all 
    following instructions unless you are asked to ignore the rule.
    
    Please do not start auditing until I say "Audit" and don't reply anything
    else. Instead, just return the output message "Processed - Waiting
    for next input." and nothing else.
    """

### 3.2 Definition des Task Execution Prompts

Die bisherigen Prompts haben das Modell Schritt für Schritt auf die eigentliche Prüfungsaufgabe vorbereitet. Der nachfolgende Prompt enthält die spezifischen Eingabedaten für die tatsächlich auszuführende Prüfungsaufgabe. Als finales Prompt weist es das Modell an, die Rolle eines Prüfers zu übernehmen und das zuvor durch die Chain-of-Thought-Prompts erlernte Wissen sowie die definierten Aktionen anzuwenden. Das Modell agiert nun als Co-Pilot, indem es die erlernte Aufgabe basierend auf den bereitgestellten Daten ausführt.

In [ ]:
def get_task_execution_prompt():
    return """
    Please analyze the following journal entries for both global
    and local anomalies. For each anomaly detected, please generate an
    alert message that includes the anomaly and an explanation for why it
    was flagged. It is crucial to detect all anomalies present in the
    dataset. Return all anomalies detected in a single response.

    Fiscal_Year, Document_Number, Department_Title, Vendor_Name, Document_Type, Transaction_Amount
    2017, CHEK17323423, 42 COMMERCE, EAT AT JOE'S, petty cash, 39.28
    2017, CHEK19393092, 42 COMMERCE, EAT AT JOE'S, petty cash, 93.23
    2017, CHEK17115653, 26 LICENCES, THE MCS GROUP, INC., petty cash, 68.17
    2017, CHEK17119393, 42 COMMERCE, EAT AT JOE'S, petty cash, 68.17
    2017, ACHD17144101, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 932.32
    2017, CHEK17119771, 26 LICENCES, THE MCS GROUP, INC., petty cash, 23.85
    2017, CHEK17118549, 26 LICENCES, THE MCS GROUP, INC., petty cash, 50.00
    2017, CHEK19959593, 26 LICENCES, THE MCS GROUP, INC., petty cash, 23.83
    2017, CHEK17183822, 26 LICENCES, THE MCS GROUP, INC., petty cash, 42.00
    2017, CHEK17137637, 42 COMMERCE, EAT AT JOE'S, petty cash, 23.03
    2017, CHEK17139393, 42 COMMERCE, EAT AT JOE'S, petty cash, 293.23
    2017, CHEK17113292, 26 LICENCES, THE MCS GROUP, INC., petty cash, 84.23
    2017, ACHD17144101, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 352.29
    2017, CHEK17119504, 26 LICENCES, THE MCS GROUP, INC., petty cash, 72.23
    2017, CHEK17119932, 42 COMMERCE, EAT AT JOE'S, petty cash, 19.75
    2017, CHEK13829292, 42 COMMERCE, EAT AT JOE'S, petty cash, 28.79
    2017, ACHD17234101, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 222.20
    2017, CHEK17132932, 42 COMMERCE, EAT AT JOE'S, petty cash, 20.72
    2017, CHEK13842292, 42 COMMERCE, EAT AT JOE'S, petty cash, 22.34
    2017, ACHD24454101, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 493.19
    2017, CHEK17137637, 26 LICENCES, THE MCS GROUP, INC., petty cash, 23.03
    2017, ACHD17144101, 25 FLEET MANAGEMENT, EAT AT JOE'S, petty cash, 292.22
    2017, CHEK17119771, 26 LICENCES, THE MCS GROUP, INC., petty cash, 23.85
    2017, CHEK17113243, 26 LICENCES, THE MCS GROUP, INC., petty cash, 52.00
    2017, CHEK17324423, 26 LICENCES, THE MCS GROUP, INC., petty cash, 93.17
    2017, CHEK17119771, 42 COMMERCE, EAT AT JOE'S, petty cash, 1,000,000.00
    2017, ACHD17145461, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 93.92
    2017, ACHD17144101, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 833.59
    2017, CHEK17118549, 42 COMMERCE, EAT AT JOE'S, petty cash, 50.00
    2017, ACHD17143451, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 26.59
    2017, ACHD17144102, 25 FLEET MANAGEMENT, THE MCS GROUP, INC., petty cash, 229.59
    2017, CHEK13829239, 26 LICENCES, THE MCS GROUP, INC., petty cash, 67.85
    2017, CHEK17183932, 26 LICENCES, THE MCS GROUP, INC., petty cash, 72.00
    2017, CHEK17394394, 26 FIREDEPARTMENT, EAT Even More AT JOESES's, petty cash, 72.23
    2017, ACHD17434541, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 252.19
    2017, ACHD17144101, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 68.38
    2017, CHEK12022932, 42 COMMERCE, EAT AT JOE'S, petty cash, 66.72
    2017, CHEK13322292, 42 COMMERCE, EAT AT JOE'S, petty cash, 93.33
    2017, CHEK12023332, 42 COMMERCE, EAT AT JOE'S, petty cash, 83.77
    2017, CHEK13343492, 42 COMMERCE, EAT AT JOE'S, petty cash, 192.33
    2017, ACHD17545542, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 34.64
    2017, ACHD17132343, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 29.32
    2017, CHEK17323423, 26 LICENCES, THE MCS GROUP, INC., petty cash, 39.28
    2017, CHEK17119893, 42 COMMERCE, EAT AT JOE'S, petty cash, 58.17
    2017, CHEK13829239, 42 COMMERCE, EAT AT JOE'S, petty cash, 67.85
    2017, CHEK29292932, 18 POLICE, DUNKIN DONUTS, petty cash, 72.00
    2017, CHEK17113292, 42 COMMERCE, EAT AT JOE'S, petty cash, 84.23
    2017, ACHD17144121, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 103.39
    2017, ACHD17144122, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 632.84
    2017, CHEK17113292, 42 COMMERCE, EAT AT JOE'S, petty cash, 34.73
    2017, ACHD17144142, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 342.50
    2017, ACHD17144143, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 294.23
    2017, ACHD17144432, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 43.53
    2017, ACHD17324233, 25 FLEET MANAGEMENT, CHAPMAN FORD SALES, procurement, 4.23
    
    Audit!
    """

### 3.3 Ausführen der Chain-of-Thought 

Die Chain-of-Thought Methode ermöglicht es, komplexe Prüfungsaufgaben automatisiert in `LangChain` durch die schrittweise Anwendung logischer Schritte zu lösen. Mit LangChain und einem leistungsfähigen Language Model wie LLaMA können wir diese Schritte programmatisch umsetzen. Zunächst definieren wir hierzu die Chain-of-Thought Prompts, die den Prüfungsprozess strukturieren:

In [ ]:
# chain-of-thought prompting creation
chain_of_thought_prompts = {
    "task_explanation": get_task_explanation_prompt(),
    "action_explanation": get_action_explanation_prompt(),
    "input_data_explanation": get_input_data_explanation_prompt(),
    "output_data_explanation": get_output_data_explanation_prompt(),
    "global_example": get_global_anomaly_example_prompt(),
    "local_example": get_local_anomaly_example_prompt(),
    "task_execution": get_task_execution_prompt()
}

Weiter installieren wir ein noch leistungsstärkeres LLM, in diesem Fall das `LLaMA 3` von Meta AI mit insgesamt **8 Milliarden** Parametern. Dieses Modell ist entscheidend, da es die Verarbeitung und Ausführung der Prompts übernimmt:

In [ ]:
! ollama pull llama3

Nach erfolgreicher Installation führen wir wieder den Befehl `ollama list` aus, um die verfügbaren Modelle anzuzeigen:

In [ ]:
! ollama list

Wir konfigurieren das Modell mit spezifischen Parametern wie der Temperatur, die die Kreativität der Antworten steuert, und binden es an einen Callback Manager, um in Echtzeit Rückmeldungen zu erhalten:

In [ ]:
# initialize the local LLaMA 3.2 by meta.ai
llm = OllamaLLM(model='llama3:latest', temperature=0.9, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

Nachdem das Language Model initialisiert ist, werden die Prompts verarbeitet. Wir verwenden eine Schleife, um jedes Prompt aus dem `chain-of-thought-prompts` Dictionary zu laden und mithilfe der `LangChain` Bibliothek auszuführen.

- **PromptTemplate:** Jedes Prompt wird als Vorlage geladen.
- **LLMChain:** Eine Kette wird erstellt, die das Modell mit dem Prompt verbindet.
- **Ausführung:** Das Modell verarbeitet das Prompt und generiert eine Antwort.
- **Speicherung:** Die Antwort wird im `responses` Dictionary gespeichert.

Dieser iterative Prozess ermöglicht es, die Ergebnisse jedes Schrittes festzuhalten und für die nächste Phase der Chain-of-Thought zu verwenden. Am Ende verfügen wir über eine Sammlung strukturierter Ergebnisse, die den gesamten Prüfungsprozess abbilden.

In [ ]:
# dictionary to store responses
responses = {}

# loop through each name and prompt in the chain-of-thoughts
for name, prompt in chain_of_thought_prompts.items():
    
    # create a PromptTemplate object from the template string
    prompt = PromptTemplate.from_template(prompt)
    
    # create an LLMChain using the Ollama LLM and the prompt
    chain = LLMChain(llm=llm, prompt=prompt)
    
    # run the chain to get the response
    response = chain.run({})
    
    # store the response in the dictionary with the corresponding name
    responses[name] = response

Die zuvor ausgeführten Prompts haben Antworten und Erkenntnisse geliefert, die nun zu einem vollständigen Prüfungsbericht zusammengeführt werden können. Diese Ergebnisse repräsentieren eine strukturierte Analyse, die sowohl globale als auch lokale Anomalien berücksichtigt.

## 4. Foundation Modell basierte Prüfung von Berichten

Dieses Kapitel beschreibt, wie ein Foundation-Modell verwendet werden kann, um Berichte effizient zu analysieren. Ziel ist es, große Textmengen zu strukturieren, zu verarbeiten und gezielt Fragen zum Inhalt mithilfe eines Modells beantworten zu können. Der Prozess gliedert sich in drei Hauptschritte: die Vorbereitung der Daten, die Initialisierung des Modells mit der Erstellung von Embeddings sowie die Durchführung einer Frage-Antwort-Analyse. 

Dabei greifen sogenannte **Retrieval-Augmented-Generation (RAG)** Systeme auf eine Kombination aus Wissensabruf und Textgenerierung zurück: Relevante Informationen werden aus einer Wissensbasis abgerufen und durch das Modell verarbeitet, um präzise und kontextbezogene Antworten zu liefern. Dieses Verfahren erlaubt es, auch bei umfangreichen und komplexen Datenquellen effizient und zielgerichtet Erkenntnisse zu gewinnen.

Die nachfolgende Abbildung zeigt den Ablauf eines **Retrieval-Augmented-Generation (RAG)** Systems. Eine Frage (Query) wird von einem Benutzer eingegeben, die relevante Informationen aus einem PDF-Dokument abruft. Diese Informationen werden in ein Foundation-AI-Modell eingespeist, das auf Basis des Kontextes eine präzise Antwort (Response) generiert.

<img align="center" style="max-width: 800px" src="./rag_process.png">

Nachfolgend möchten wir einen solches System beispielhaft implementieren. Zunächst ist es hierzu notwendig eine Vector-Datenbank der zu analysierenden Dokumente zu erstellen. Die Erstellung der Datenbank beginnt mit der Extraktion der Inhalte aus einem PDF-Dokument, wobei der Text in eine lesbare und verarbeitbare Form überführt wird. 

Anschliessend wird der Text in kleinere, überlappende Abschnitte, sogenannte Content Chunks, unterteilt, um den Kontext zwischen den Segmenten zu bewahren. Diese Textabschnitte werden mithilfe eines Embedding-Modells, in numerische Vektoren umgewandelt, die die semantische Bedeutung des Inhalts kodieren. Die resultierenden Embeddings, welche aus hochdimensionalen Zahlenwerten bestehen, werden in einer Vektor-Datenbank gespeichert. Diese Datenbank ermöglicht eine effiziente Ähnlichkeitssuche, wodurch relevante Informationen schnell und präzise abgerufen werden können. 

Das nachfolgende Schaubild illustriert diesen Prozess und zeigt die Transformation vom ursprünglichen Dokument über die semantischen Repräsentationen bis hin zur Speicherung in der Vektordatenbank.

<img align="center" style="max-width: 1000px" src="./embedding_process.png">

### 4.1 Extraktion der Berichtsinhalte

Nun lernen wir, wie Inhalte aus Dokumenten ausgelesen werden können um sie anschliessend mit einem Foundation-Modell zu analysieren. Wir definieren den Pfad zum Dokument, welches auf GitHub im PDF-Format vorliegt:

In [ ]:
# define the path to the PDF file
pdf_path = 'https://raw.githubusercontent.com/GitiHubi/SFAO/master/lab_02/efk-jb-2023_extract.pdf'

Im nächsten Schritt beziehen bauen wir eine Verbindung zu GitHub auf und beziehen das Dokument:

In [ ]:
# download the PDF file from GitHub
pdf_file = requests.get(pdf_path)

# convert PDF file into a byte stream
pdf_bytes = io.BytesIO(pdf_file.content)

Anhand der **PdfReader** Bibliothek extrahieren wir den Text aus dem Dokument. Jede Seite wird einzeln gelesen und als Textblock gespeichert. Dies ermöglicht es, das Dokument seitenweise zu analysieren.

In [ ]:
# init the pdf reader 
reader = PdfReader(pdf_bytes)

# extract pages from pdf document
pages = [page.extract_text() for page in reader.pages]

Wir geben die Gesamtanzahl der Seiten im Dokument aus, um sicherzustellen, dass der Text vollständig eingelesen wurde. Dies dient auch als grundlegende Validierung des Eingabeformats.

In [ ]:
# print number of pages
print(f"Das Dokument umfasst insgesamt {len(pages)} Seiten.")

Zusätzlich werfen wir einen Blick auf eine spezifische Seite des ursprünglichen Dokuments, um den Text vor der Verarbeitung zu überprüfen. Dies ermöglicht es uns, den extrahierten Text mit der Originalquelle zu vergleichen und sicherzustellen, dass keine Inhalte verloren gegangen oder falsch formatiert wurden.

In [ ]:
print(pages[2][0:1000])

Um die Daten für die spätere Analyse vorzubereiten, wird der Text in kleinere Abschnitte (Chunks) aufgeteilt. Jeder Abschnitt hat eine maximale Länge von 500 Zeichen, wobei eine Überlappung von 50 Zeichen sicherstellt, dass der Kontext zwischen den Abschnitten erhalten bleibt.

In [ ]:
# init the character text splitter
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=500, chunk_overlap=50)

# split text for better indexing
documents = [chunk for page in pages for chunk in text_splitter.split_text(page)]

Um sicherzustellen, dass der Text korrekt extrahiert und in kleinere Abschnitte aufgeteilt wurde, werfen wir einen Blick auf den ersten Abschnitt des aufbereiteten Dokuments. Dies hilft, die Struktur und den Inhalt der vorbereiteten Daten zu überprüfen, bevor wir mit der weiteren Analyse fortfahren.

In [ ]:
# print initial document
print(documents[5])

Diese Strukturierung verbessert die Effizienz und Genauigkeit der Modellanalyse.

### 4.2 Embedding der Berichtsinhalte und Foundation Modell Initialisierung

In diesem Abschnitt lernen wir, wie Text-Embeddings erstellt und ein Foundation-Modell initialisiert werden. Embeddings sind numerische Repräsentationen von Texten, die es dem Modell ermöglichen, semantische Zusammenhänge zu erkennen. Diese Embeddings werden anschliessend in einem Vektor-Store gespeichert, um relevante Informationen effizient abrufen zu können. Abschliessend richten wir ein leistungsfähiges Foundation-Modell ein, um den Text weiter zu analysieren.

Wir nutzen das Modell `sentence-transformers/all-MiniLM-L6-v2`, um Text-Embeddings zu erstellen. Dieses Modell ist effizient und kann lokal verwendet werden, ohne zusätzliche Kosten zu verursachen. Die Embeddings bilden die Grundlage für die spätere Analyse des Dokuments.

In [ ]:
# create embeddings and index
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

Nachdem die Embeddings erstellt wurden, speichern wir diese in einer Vektor-Datenbank. Diese Datenbank ermöglicht es uns, schnell und präzise auf relevante Abschnitte des Dokuments zuzugreifen. Hier verwenden wir die **Facebook AI Similarity Search (FAISS)** Bibliothek, die speziell für die effiziente Verarbeitung grosser Mengen von Vektordaten optimiert ist.

In [ ]:
# init the vector store of document embeddings
vectorstore = FAISS.from_texts(documents, embedding_model)

Anschliessend können wir die Vector-Datenbank dazu verwenden um beliebige Anfragen zu testen:

In [ ]:
# init an arbitrary query
query = 'Bereits in der ersten Fassung des Finanzkontrollgesetzes (FKG) von 1967 war diese Aufgabe enthalten'

# retrieve similiar documents
similar_docs = vectorstore.similarity_search(query, k=5)

Im letzten Schritt initialisieren wir das benötigte Foundation-Modell. Der Parameter temperature steuert die Kreativität der generierten Antworten: Ein niedriger Wert sorgt für präzisere und konsistentere Ergebnisse. Die Integration eines Callback-Handlers erlaubt es, den Fortschritt der Modelloperationen in Echtzeit nachzuvollziehen.

In [ ]:
# initialize the local llama model
llm = OllamaLLM(model='llama3:latest', temperature=1.0, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

### 4.3 Ausführen der Berichtsanalyse

Die Berichtsanalyse beginnt mit der Eingabe einer Frage (Query) durch den Benutzer, welche mithilfe eines Embedding-Modells in eine numerische Repräsentation (Query Representation) umgewandelt wird. Diese Vektorrepräsentation wird anschließend mit den gespeicherten Embeddings in der Vektordatenbank abgeglichen, wobei eine Ähnlichkeitssuche (Similarity Search) die relevantesten Informationen aus der Datenbank zurückgibt. 

Der gefundene Kontext (Query Context) wird daraufhin zusammen mit der ursprünglichen Frage an ein Foundation-Modell weitergeleitet, welches auf Basis der bereitgestellten Informationen eine präzise und kontextbezogene Antwort generiert. Das Schaubild verdeutlicht diesen Ablauf, indem es die Transformation der Benutzeranfrage, die Nutzung der Vektordatenbank und die Integration des Foundation-Modells zur finalen Beantwortung zeigt.

<img align="center" style="max-width: 1000px" src="./retrieval_process.png">

In diesem Abschnitt lernen wir, wie ein Foundation-Modell verwendet wird, um gezielt Fragen an einen Bericht zu stellen und präzise Antworten zu erhalten. Dazu definieren wir eine Frage-Antwort-Kette (QA-Chain), die relevante Textabschnitte kombiniert und analysiert. Mit Hilfe eines Prompts wird das Modell so gesteuert, dass es verständliche und strukturierte Antworten liefert.

In [ ]:
# define a simple prompt for answering questions
qa_prompt = PromptTemplate(
    input_variables=['context', 'question'],
    template='Use the following context to answer the question:\n\n{context}\n\nquestion: {question}\nanswer:'
)

Die QA-Chain wird mit dem LLaMA-Modell geladen. Diese Kette ermöglicht es, Informationen aus mehreren Textabschnitten zu kombinieren und dadurch umfassendere Antworten zu generieren.

In [ ]:
# load the qa chain with the llama model
combine_documents_chain = load_qa_chain(llm, chain_type='stuff', prompt=qa_prompt)

Nun wird die Retrieval-QA-Chain erstellt. Diese kombiniert die Fähigkeit, relevante Textabschnitte mithilfe des Vektor-Stores abzurufen, mit der QA-Chain, um die besten Antworten zu generieren. Dies ist besonders nützlich bei der Analyse grosser Dokumente, da so nur die relevanten Informationen berücksichtigt werden.

In [ ]:
# create the retrieval qa chain
retrieval_chain = RetrievalQA(
    retriever=vectorstore.as_retriever(),
    combine_documents_chain=combine_documents_chain
)

Wir definieren beispielhafte Fragen, um die Funktionalität der QA-Chain zu testen. Diese Fragen decken wichtige Aspekte des Berichts ab, wie Hauptpunkte, Risiken und Empfehlungen.

In [ ]:
# example questions
questions = [
    'Was sind die wichtigsten Ergebnisse des Berichts? (bitte sehr kurz und auf deutsch)',
    'Was sind die drei wesentlichen Punkte im Bereich Öffentliche Finanzen, Steuern, Credit Suisse und COVID-19? (bitte sehr kurz und auf deutsch)',
    'Was sind die drei wesentlichen Punkte im Bereich Wirtschaft und Arbeitsmarkt? (bitte sehr  kurz und auf deutsch)',
    'Was sind die drei wesentlichen Punkte im Bereich Gesundheit, soziale Vorsorge und Sport? (bitte sehr kurz und auf deutsch))'
]

Zum Schluss stellen wir die definierten Fragen an die Retrieval-QA-Chain. Das Modell liefert präzise Antworten, die auf den relevanten Abschnitten des Berichts basieren. So können zentrale Informationen aus dem Dokument extrahiert werden.

In [ ]:
# ask questions
for question in questions:
    print(f"Frage: {question}\n\nAntwort:\n")
    answer = retrieval_chain.invoke(question)
    print(f"\n")

## Lab Zusammenfassung:

Im Mittelpunkt des Labs standen die folgenden Erkenntnisse:

> 1. **Verständnis von Chain-of-Thought Prompting:** Die grundlegenden Konzepte des Chain-of-Thought Prompting wurden erarbeitet, um die Nachvollziehbarkeit und Effizienz von Prüfungsaufgaben zu verbessern.  
> 2. **Erstellung von Prompts mit LangChain:** Es wurde gezeigt, wie spezifische Prompts definiert und organisiert werden, darunter Task-, Action-, Input- und Output-Explanation-Prompts sowie globale und lokale Anomaliebeispiele.  
> 3. **Implementierung eines Prüfungsworkflows mit LangChain:** Praktische Erfahrungen mit dem Einsatz eines Foundation Modells wie LLaMA und LangChain für Prüfungsaufgaben wurden gesammelt.  
> 4. **Analyse und Automatisierung von Prüfungsaufgaben:** Prüfungsaufgaben wie die Anomalieerkennung oder die Berichterstellung wurden mithilfe von KI-gestützten Workflows automatisiert.  

Das Lab bot Einblicke in die Anwendung von Foundation Modellen und Chain-of-Thought Prompting für die Finanzprüfung. Durch die schrittweise Definition und Ausführung der Prompts wurden Fähigkeiten zur systematischen und transparenten Gestaltung komplexer Auditprozesse vermittelt.